In [2]:
import pandas as pd
import numpy as np

from src.methods.CAA_class import _CAA
from src.methods.OAA_class import _OAA
from src.methods.RBOAA_class import _RBOAA

from src.utils.synthetic_data_class import _synthetic_data
from src.utils.synthetic_data_naive import synthetic_data_naive

from src.utils.eval_measures import NMI, MCC

### Loading in previously generated synthetic data

In [7]:
X_1000 = np.load('SyntheticData/1000_respondents/X.npy') # questionnaire data
A_1000 = np.load('SyntheticData/1000_respondents/A.npy') # weighting of respondents on archetypes
Z_1000 = np.load('SyntheticData/1000_respondents/Z.npy') # archetype matrix (X @ B)

In [10]:
print(f'X: {X_1000.shape}')
print(f'A: {A_1000.shape}') # [n_archetypes x n_respondents]
print(f'Z: {A_1000.shape}')

X: (20, 1000)
A: (3, 1000)
Z: (3, 1000)


### Perform analysis with RBOAA

In [12]:
RBOAA = _RBOAA() # instantiate
rboaa_result = RBOAA._compute_archetypes(X=X_1000,
                          K=3, # how many archetypes to use for analysis
                          p=5, # length of likert scale in X
                          n_iter= 15000, # number of gradient steps
                          lr=0.01,
                          mute=False, # verbosity
                          early_stopping=True, # whether to terminate if loss does not improve
                          with_OAA_initialization=False, # whether to hotstart hyperparameters from OAA analysis
                          beta_regulators=False, # whether to use c1 and c2 as additional hyperparameters to model ordinal scale arbitrarily
                          alternating=False, # whether to first train with a low (and capped) sigma
                          columns=[f'q{i}' for i in range(X_1000.shape[0])]
                          )


Response Bias Ordinal Archetypal Analysis in progress...
Finished Successfully after 3.4 seconds!                               


Analysis ended due to early stopping.

/////////////// INFORMATION ABOUT RESPONSE BIAS ORDINAL ARCHETYPAL ANALYSIS \\\\\\\\\\\\\\\\
▣ The Response Bias Ordinal Archetypal Analysis was computed using 3 archetypes.
▣ The Response Bias Ordinal Archetypal Analysis was computed on 20 attributes.
▣ The Response Bias Ordinal Archetypal Analysis was computed on 1000 subjects.
▣ The Response Bias Ordinal Archetypal Analysis ran for 2201 iterations.
▣ The Response Bias Ordinal Archetypal Analysis took 3.44 seconds to complete.
▣ The final loss was: 2.133932590484619.


In [13]:
### Extract A and Z matrices from RBOAA result object and compute NMI and MCC
print(f'NMI: {NMI(rboaa_result.A, A_1000)}')
print(f'MCC: {MCC(rboaa_result.Z, Z_1000)}')

NMI: 0.8879897047419514
MCC: 0.9974331151366035


Closing remarks:
* The other methods e.g. OAA and RBOAA also have a _compute_archetypes method which creates a result object.
* The parameters to OAA._compute archetypes are identical to that of RBOAA
* We set up a wrapper class (inside AAM.py). I will make the new changes compatible with that interface before publication - it is not critical for generating results but only helps the user experience.